In [ ]:
import pandas
import numpy as np
from io import open
import os
import csv

In [ ]:
def get_chunk_container():
    return pandas.read_csv('assets/twcs/twcs.csv', sep=',',chunksize=1000, dtype={'in_response_to_tweet_id': "string", "response_tweet_id":"string"})


In [ ]:
"""
- Get all the responses
- find the tweet that the response corisponds to.
pairs = {
    "question_id" : "response_id"
}
"""
chunk_container = get_chunk_container()
ID_pairs = dict()

for chunk in chunk_container:
    rnan = chunk[pandas.notnull(chunk["in_response_to_tweet_id"])]
    for index, row in rnan.iterrows():
        if row.author_id == "AppleSupport":
            ID_pairs[row.tweet_id] = row.in_response_to_tweet_id

In [ ]:
def ID_to_tweet(key, value):
    chunk_container = get_chunk_container()
    response = question = pandas.Series(dtype='string')
    for chunk in chunk_container:
        if response.empty:
            response = chunk.loc[chunk.tweet_id.isin([str(key)])]
        if question.empty:
            question = chunk.loc[chunk.tweet_id.isin([str(value)])]

        if not response.empty and not question.empty:
            break # If both tweets have been found
    if response.empty or  question.empty:
            print("No values exist")
            return None
    return {question.text.iloc[0], response.text.iloc[0]}

In [ ]:
def saveToFile(pairs, fileDir):
    with open(fileDir, "w", encoding='utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter='\t', lineterminator='\n')
        for key, value in pairs.items():
            val = ID_to_tweet(key, value)
            if val == None:
                continue
            else:
                writer.writerow(val)

In [ ]:
ID_to_tweet(list(ID_pairs.items())[0][0],list(ID_pairs.items())[0][1])

In [ ]:
fileDir = os.path.join('assets', 'AppleSupport.txt')
saveToFile(ID_pairs, fileDir)